In [1]:
import glob, gzip, os
import numpy as np
import tensorflow as tf
np.set_printoptions(linewidth=200)

2024-02-13 10:01:17.414370: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-13 10:01:17.634644: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-13 10:01:17.634699: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-13 10:01:17.669428: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-13 10:01:17.748845: I tensorflow/core/platform/cpu_feature_guar

In [9]:
from datasets import load_dataset

dataset = load_dataset("jeremygf/domains-app-alpha")
dataset = dataset['train']
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 534152
})

In [13]:
class LayerNorm(tf.keras.layers.Layer):
    def __init__(self, epsilon=1e-6, **kwargs):
        super(LayerNorm, self).__init__(**kwargs)
        self.epsilon = epsilon

    def build(self, input_shape):
        self.gamma = self.add_weight(name='gamma', shape=input_shape[-1:],
                                     initializer='ones', trainable=True)
        self.beta = self.add_weight(name='beta', shape=input_shape[-1:],
                                    initializer='zeros', trainable=True)
        super(LayerNorm, self).build(input_shape)

    def call(self, x):
        mean = tf.reduce_mean(x, axis=-1, keepdims=True)
        variance = tf.reduce_mean(tf.square(x - mean), axis=-1, keepdims=True)
        x = (x - mean) * tf.math.rsqrt(variance + self.epsilon)
        x = self.gamma * x + self.beta
        return x


class CausalSelfAttention(tf.keras.layers.Layer):
    def __init__(self, num_heads, embedding_dim):
        super().__init__()

        self.mha = tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads, 
            key_dim=embedding_dim
        )

        self.layer_norm = LayerNorm()
    
    def call(self, x):
        attn_output = self.mha(
            query=x, 
            value=x, 
            use_causal_mask=True
        )
        x = x + attn_output
        x = self.layer_norm(x)
        return x
    
class FeedForward(tf.keras.layers.Layer):
    def __init__(self, embedding_dim, feedforward_dim, dropout_rate=0.1):
        super().__init__()
        
        self.d1 = tf.keras.layers.Dense(feedforward_dim, activation='gelu')
        self.d2 = tf.keras.layers.Dense(embedding_dim)
        self.dropout = tf.keras.layers.Dropout(dropout_rate)

        self.layer_norm = LayerNorm()

    def call(self, x):
        x = self.d2(self.d1(x))
        x = x + self.dropout(x)
        x = self.layer_norm(x) 
        return x

class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self, embedding_dim, num_heads, feedforward_dim, dropout_rate=0.1):
    super(DecoderLayer, self).__init__()

    self.causal_self_attention = CausalSelfAttention(num_heads=num_heads, embedding_dim=embedding_dim)

    self.ffn = FeedForward(embedding_dim=embedding_dim, feedforward_dim=feedforward_dim, dropout_rate=dropout_rate)

  def call(self, x):
    x = self.causal_self_attention(x=x)

    x = self.ffn(x)
    return x    
    
    
def positional_encoding(length, depth):
  depth = depth/2

  positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
  depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)

  angle_rates = 1 / (10000**depths)         # (1, depth)
  angle_rads = positions * angle_rates      # (pos, depth)

  pos_encoding = np.concatenate(
      [np.sin(angle_rads), np.cos(angle_rads)],
      axis=-1) 

  return tf.cast(pos_encoding, dtype=tf.float32)

class PositionalEmbedding(tf.keras.layers.Layer):
  def __init__(self, vocab_size, embedding_dim):
    super().__init__()
    self.embedding_dim = embedding_dim
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True) 
    self.pos_encoding = positional_encoding(length=MAX_SEQ_LEN, depth=embedding_dim)

  def compute_mask(self, *args, **kwargs):
    return self.embedding.compute_mask(*args, **kwargs)

  def call(self, x):
    length = tf.shape(x)[1]
    x = self.embedding(x)
    # This factor sets the relative scale of the embedding and positonal_encoding.
    x *= tf.math.sqrt(tf.cast(self.embedding_dim, tf.float32))
    x = x + self.pos_encoding[tf.newaxis, :length, :]
    return x

class CharTransformer(tf.keras.Model):
    def __init__(self, vocab_size: int, num_layers: int, embedding_dim: int, 
        num_heads: int, intermediate_dim: int) -> None:

        super().__init__()

        self.embeddings = PositionalEmbedding(vocab_size, embedding_dim)

        self.decoders = [
            DecoderLayer(embedding_dim, num_heads, intermediate_dim)
            for _ in range(num_layers)
        ]

        self.dense = tf.keras.layers.Dense(vocab_size, activation='relu')
        
    def call(self, inputs):
        x = self.embeddings(inputs)  

        for layer in self.decoders:
            x = layer(x)
        return self.dense(x)       

In [14]:
@tf.function
def masked_loss(label, pred):
    mask = label != 0
    loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')
    loss = loss_object(label, pred)

    mask = tf.cast(mask, dtype=loss.dtype)
    loss *= mask

    loss = tf.reduce_sum(loss)/tf.reduce_sum(mask)
    return loss

@tf.function
def masked_accuracy(label, pred):
    pred = tf.argmax(pred, axis=2)
    label = tf.cast(label, pred.dtype)
    match = label == pred

    mask = label != 0

    match = match & mask

    match = tf.cast(match, dtype=tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    return tf.reduce_sum(match)/tf.reduce_sum(mask)

In [16]:
BATCH = 256
EPOCHS = 10

from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(
    tokenizer=tokenizer, 
    return_tensors="tf"
)

tf_train = ddict['train'].to_tf_dataset(
    columns="input_ids",
    label_cols="target_ids",
    collate_fn=data_collator,
    batch_size=BATCH,
    shuffle=True
)

tf_test = ddict['test'].to_tf_dataset(
    columns="input_ids",
    label_cols="target_ids",
    collate_fn=data_collator,
    batch_size=BATCH,
    shuffle=True
)


2024-02-13 10:02:11.010343: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-13 10:02:11.125084: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-13 10:02:11.125120: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-13 10:02:11.127840: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-13 10:02:11.127873: I external/local_xla/xla/stream_executor

In [17]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super().__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    step = tf.cast(step, dtype=tf.float32)
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

  def get_config(self):
    return {
      "d_model": self.d_model,
      "warmup_steps": self.warmup_steps
    }

In [18]:

EMBED_DIM = 256

NUM_HEADS = 4
FEED_FORWARD_DIM = 512
NUM_LAYERS = 2



learning_rate = CustomSchedule(EMBED_DIM)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

model = CharTransformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    embedding_dim=EMBED_DIM,
    num_heads=NUM_HEADS,
    intermediate_dim=FEED_FORWARD_DIM
)

model.compile(loss=masked_loss, metrics=[masked_accuracy], optimizer=optimizer)

2024-02-13 10:06:07.704767: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


In [88]:
tf.debugging.set_log_device_placement(False)

In [19]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [20]:
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [22]:
model.fit(
    tf_train, 
    validation_data=tf_test, 
    epochs=EPOCHS,
    callbacks=[tensorboard_callback] 
)

Epoch 1/10
1670/1670 [==============================] - 100s 60ms/step - loss: 4.1581 - masked_accuracy: 0.2881 - val_loss: 4.2989 - val_masked_accuracy: 0.2769
Epoch 2/10
1670/1670 [==============================] - 95s 57ms/step - loss: 4.1394 - masked_accuracy: 0.2898 - val_loss: 4.3002 - val_masked_accuracy: 0.2765
Epoch 3/10
1670/1670 [==============================] - 96s 57ms/step - loss: 4.1227 - masked_accuracy: 0.2918 - val_loss: 4.3028 - val_masked_accuracy: 0.2766
Epoch 4/10
1670/1670 [==============================] - 97s 58ms/step - loss: 4.1068 - masked_accuracy: 0.2933 - val_loss: 4.3052 - val_masked_accuracy: 0.2768
Epoch 5/10
1670/1670 [==============================] - 98s 59ms/step - loss: 4.0917 - masked_accuracy: 0.2953 - val_loss: 4.3071 - val_masked_accuracy: 0.2766
Epoch 6/10
1670/1670 [==============================] - 92s 55ms/step - loss: 4.0779 - masked_accuracy: 0.2968 - val_loss: 4.3104 - val_masked_accuracy: 0.2758
Epoch 7/10
1670/1670 [=================

In [66]:
model.save("char_transformer.keras")

NotImplementedError: Learning rate schedule 'CustomSchedule' must override `get_config()` in order to be serializable.

In [63]:
prompt = ''
prompt_length = len(tokenizer.encode(prompt))-1
print(prompt_length)
prompt = tokenizer.encode(prompt, padding='max_length', max_length=MAX_SEQ_LEN, return_tensors='tf')
prompt

1


<tf.Tensor: shape=(1, 20), dtype=int32, numpy=array([[1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>

In [60]:
def nextt(prompt, cache, index):
    logits = model(prompt)[:, index-1, :]
    # Ignore hidden states for now; only needed for contrastive search.
    hidden_states = None
    return logits, hidden_states, cache

In [64]:
import keras_nlp
sampler = keras_nlp.samplers.TopPSampler(0.5)
output_tokens = sampler(
    next=nextt,
    prompt=prompt,
    index=prompt_length,  # Start sampling immediately after the [BOS] token.
)[0]
print(output_tokens)
print(tokenizer.decode(output_tokens).split('[END]'))

ModuleNotFoundError: No module named 'keras_nlp'